In [ ]:
import gzip
import json

def read_jsonl_gz(file_path):
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            yield json.loads(line)

file_path = './data/manifests/genshin_supervisions_dev.jsonl.gz'

# 逐行读取和解析 JSON 数据
json_data = list(read_jsonl_gz(file_path))

# 查看数据
json_data


In [1]:
import os
from bin.myinfer import InferModel

os.environ["CUDA_VISIBLE_DEVICES"] = "2"
model = InferModel(checkpoint="exp/valle1/best-valid-loss.pt")

In [38]:
import os
import json
import shutil
from typing import List
from collections import OrderedDict
from os import PathLike
from pathlib import Path

import sys

# 清除导入的缓存
if 'bin.utils' in sys.modules:
    del sys.modules['bin.utils']

from bin.utils import *


class GenShinDataWrapper:
    def __init__(
        self,
        dir: PathLike = Path("download/genshin"),
    ) -> None:
        self.voice = json.load(open(dir / "result_chs.json"))
        self.voice_test = json.load(open(dir / "result_chs_test.json"))

        # 按照 value['npcName'] 进行分组
        grouped_dict = OrderedDict()
        for key, value in self.voice.items():
            group_key = value["npcName"]
            grouped_dict.setdefault(group_key, []).append(key)
        self.grouped_dict = grouped_dict
        self.text_processor = TextProcessor()

    def create_file(
        self,
        file_name: PathLike,
        ids: List[str],
        texts: List[str],
        infer_dir: PathLike,
        copy: bool = False,
    ):
        os.makedirs(infer_dir, exist_ok=True)
        with open(file_name, "w") as file:
            used = {}
            for n, id in enumerate(ids):
                value = self.voice[id]
                text_prompt = value["text"]
                audio_prompt = f'download/genshin/{value["fileName"]}'
                text = self.text_processor.text_pre_process(texts[n % len(texts)])
                if id in used:
                    used[id] += 1
                    suffix = f"infer{used[id]}"
                else:
                    used[id] = 0
                    suffix = "infer"
                audio_out = f"{infer_dir}/{id}_{suffix}.wav"
                file.write(
                    "\t".join([text_prompt, audio_prompt, text, audio_out]) + "\n"
                )
        print(open(file_name).read())
        if copy:
            shutil.copyfile(file_name, f"../../../{file_name}")
        return file_name

    def get_npc_ids(
        self,
        name: str,
        count: int = 5,
        min_len: int = 8,
        max_len: int = 18,
    ):
        ids = [
            key
            for key in self.grouped_dict[name]
            if min_len <= len(self.voice[key]["text"]) <= max_len
        ]
        return random_samples(ids, count)

    def get_test_ids(
        self,
        count: int = 5,
        min_len=8,
        max_len=18,
    ) -> List[str]:
        ids = [
            key
            for key, value in self.voice_test.items()
            if min_len <= len(value["text"]) <= max_len
        ]
        return random_samples(ids, count)

In [43]:
wrapper = GenShinDataWrapper()

ids = wrapper.get_npc_ids("胡桃")
# ids = ["c15260d94fa350c1"] * 5
# ids = wrapper.get_test_ids(5)

texts = get_tts_texts("hutao")

file_name = wrapper.create_file("genshin.txt", ids, texts, "audios/genshin")

只要我不忙,随时欢迎你来哦.	download/genshin/Chinese/VO_HS/VO_hutao/vo_hutao_mimitomo_friendship1_03b_2.wav	相信自己的力量,勇往直前,你一定能够战胜一切困难.	audios/genshin/b64bd7f1fdc737f1_infer.wav
再见,今后有机会一起写诗!	download/genshin/Chinese/VO_EQ/VO_hutao/vo_EQHDJ201_4_talk_hutao_05.wav	冒险的旅程中,希望我们能够结识更多的朋友,共同创造美好的回忆.	audios/genshin/ad5d31272e8c67a1_infer.wav
嘻嘻,月亮出来喽 咱也出门吧.	download/genshin/Chinese/VO_friendship/VO_hutao/vo_hutao_dialog_greetingNight.wav	勇敢面对挑战,超越自己的极限,你会发现无限的可能.	audios/genshin/123a131a1021960e_infer.wav
吃饱喝饱,一路走好!	download/genshin/Chinese/VO_gameplay/VO_hutao/vo_hutao_battle_skill3_01.wav	让我们砥砺前行,继续书写属于我们的传奇故事.	audios/genshin/1418e3d64adc958b_infer.wav
不如去我那儿喝杯茶?嘿	download/genshin/Chinese/VO_friendship/VO_hutao/vo_hutao_explore_idle_01.wav	用笑容面对生活的挑战,传递快乐和温暖给身边的人.	audios/genshin/44d330f3c6b8ad88_infer.wav



In [ ]:
# file_name = 'test.txt'

model.infer_by_file(file_name)

show_audios(file_name)